In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import combinations
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures

In [2]:
data = pd.read_csv('clean_housing.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)

# Build a Baseline Model
In order to fit a baseline model for use in comparisons with the more complete model I will conduct these steps:
1. Handle Muliticolinearity
2. Standardize Variables
3. Fit and Validate the Baseline Model

## 1. Handle Multicolinearity 

In [3]:
high_corr = ((abs(data.corr())> .8).sum()>1)
high_corr

price              False
bedrooms           False
bathrooms          False
sqft_living         True
floors             False
view               False
grade              False
sqft_above          True
sqft_basement      False
lat                False
long               False
sqft_living15      False
WaterFront         False
month_sold         False
age                 True
yrs_reno            True
Renovated          False
dist_to_Seattle    False
Bellevue           False
Federal Way        False
Kent               False
Renton             False
Seattle            False
rel_size           False
dtype: bool

In [4]:
data[['sqft_living', 'sqft_above', 'age', 'yrs_reno']].corr()

,sqft_living,sqft_above,age,yrs_reno
sqft_living,1.000000,0.839924,-0.366368,-0.376163
sqft_above,0.839924,1.000000,-0.483546,-0.482755
age,-0.366368,-0.483546,1.000000,0.938759
yrs_reno,-0.376163,-0.482755,0.938759,1.000000


To avoid multicolinearity issues I will drop sqft_above, because the information already exists in a combination of sqft_living and sqft_basement. I will also drop yrs_reno, because the data on whether the house has been renovated or not already exists in the renovated column. 

In [5]:
data.drop(['sqft_above', 'yrs_reno'], axis = 1, 
          inplace = True)

## 2. Standardize Variables 
To standardize the data I'll be using scikit learn's Robust Scalar function because of the existence of significant outliers in the independent variable, price.

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16262 entries, 0 to 16261
Data columns (total 22 columns):
price              16262 non-null float64
bedrooms           16262 non-null int64
bathrooms          16262 non-null float64
sqft_living        16262 non-null int64
floors             16262 non-null float64
view               16262 non-null float64
grade              16262 non-null int64
sqft_basement      16262 non-null int64
lat                16262 non-null float64
long               16262 non-null float64
sqft_living15      16262 non-null int64
WaterFront         16262 non-null int64
month_sold         16262 non-null int64
age                16262 non-null int64
Renovated          16262 non-null int64
dist_to_Seattle    16262 non-null float64
Bellevue           16262 non-null int64
Federal Way        16262 non-null int64
Kent               16262 non-null int64
Renton             16262 non-null int64
Seattle            16262 non-null int64
rel_size           16262 non-null flo

In [7]:
#Separate data into categorical and continuous groups. 
cat_data = data[['WaterFront', 'Bellevue', 'Federal Way',
                 'Kent', 'Renton', 'Seattle', 'Renovated']]

con_data = data.drop(['WaterFront', 'Bellevue', 
                      'Federal Way', 'Kent', 'Renton', 
                      'Seattle', 'Renovated'], 
                     axis=1)
con_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16262 entries, 0 to 16261
Data columns (total 15 columns):
price              16262 non-null float64
bedrooms           16262 non-null int64
bathrooms          16262 non-null float64
sqft_living        16262 non-null int64
floors             16262 non-null float64
view               16262 non-null float64
grade              16262 non-null int64
sqft_basement      16262 non-null int64
lat                16262 non-null float64
long               16262 non-null float64
sqft_living15      16262 non-null int64
month_sold         16262 non-null int64
age                16262 non-null int64
dist_to_Seattle    16262 non-null float64
rel_size           16262 non-null float64
dtypes: float64(8), int64(7)
memory usage: 1.9 MB


In [8]:
def scale(col):
    return(con_data[col]-
           con_data[col].mean())/con_data[col].std()

In [9]:
scaled_data = pd.DataFrame([])
for col in con_data.columns:
    scaled_data[col] = scale(col)

In [10]:
#Join continuous and categorical data. 
model_data = scaled_data.join(cat_data, how='outer')
model_data.describe()

,price,bedrooms,bathrooms,sqft_living,floors,view,grade,sqft_basement,lat,long,...,age,dist_to_Seattle,rel_size,WaterFront,Bellevue,Federal Way,Kent,Renton,Seattle,Renovated
count,1.626200e+04,1.626200e+04,1.626200e+04,1.626200e+04,1.626200e+04,1.626200e+04,1.626200e+04,1.626200e+04,1.626200e+04,1.626200e+04,...,1.626200e+04,1.626200e+04,1.626200e+04,16262.000000,16262.000000,16262.000000,16262.000000,16262.000000,16262.000000,16262.000000
mean,1.396415e-16,-1.339518e-15,1.764874e-15,2.311656e-16,5.501257e-14,-2.082494e-15,1.313944e-15,-4.521070e-15,-2.034863e-12,3.888316e-12,...,-2.037930e-15,-4.419537e-15,-1.550624e-14,0.000615,0.253351,0.074284,0.126491,0.159697,0.386176,0.027364
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,0.024791,0.434944,0.262240,0.332412,0.366336,0.486887,0.163148
min,-2.000057e+00,-2.761373e+00,-2.207734e+00,-2.194753e+00,-8.677580e-01,-2.329389e-01,-4.761209e+00,-6.559474e-01,-2.639059e+00,-1.915580e+00,...,-1.497173e+00,-1.631764e+00,-2.870654e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-7.874089e-01,-3.046371e-01,-7.224393e-01,-7.500213e-01,-8.677580e-01,-2.329389e-01,-4.435901e-01,-6.559474e-01,-6.177944e-01,-8.332727e-01,...,-9.359216e-01,-8.256997e-01,-5.527940e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-1.676110e-01,-3.046371e-01,2.020793e-02,-1.393616e-01,-8.677580e-01,-2.329389e-01,-4.435901e-01,-6.559474e-01,6.180182e-02,-3.080356e-01,...,-1.150707e-02,-1.701657e-01,-6.234055e-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6.374526e-01,9.237310e-01,7.628552e-01,6.202395e-01,9.616944e-01,-2.329389e-01,6.358147e-01,6.295063e-01,8.730094e-01,6.151085e-01,...,6.818038e-01,6.634144e-01,3.991267e-01,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,2.820085e+00,2.152099e+00,2.990797e+00,4.611869e+00,3.705873e+00,7.290503e+00,3.874029e+00,4.298405e+00,1.575423e+00,3.591452e+00,...,2.332544e+00,3.449029e+00,4.531018e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Build a Baseline Model

In [11]:
X = model_data.drop('price', axis=1)
y = model_data['price']

(X_train,X_test,
 y_train,y_test)=train_test_split(X,y,test_size=.2,
                                  random_state=37)

In [13]:
len(y_train)

13009

In [ ]:
linreg = LinearRegression()
model1 = linreg.fit(X_train,y_train)
crossval = KFold(n_splits = 10, shuffle = True)

In [ ]:
def report(model, ind_train, ind_test, save_as = None):
    """
    Print relevant statistics for a model.
    
    Parameters:
    model: Fitted LinearRegression object
    ind_train: independent variables for training set
    ind_test: independent variables for test set
    """
    pred_y_train = model.predict(ind_train)
    pred_y_test = model.predict(ind_test)
    
    #Print top and bottome variables by size of coefficient.
    coefs = []
    high_coefs = []
    low_coefs = []
    for i in range(0, len(model.coef_)):
        coefs.append((model.coef_[i],ind_train.columns[i]))
    for coef in coefs:
        if ((coef[0] 
            < np.quantile(model.coef_,.05))
            or (coef[0] 
            > np.quantile(model.coef_,.95))):
            high_coefs.append(coef)
    for coef in coefs:
        if ((coef[0] 
            < .05)
            and (coef[0] 
            > -.05)):
            low_coefs.append(coef)
    print('************\nHigh Impact Variables:\n')
    for variable in high_coefs:
        print('Variable: {}\nCoefficient: {}\n'
              .format(variable[1],variable[0]))
    print('************')
    print('************\nLow Impact Variables:\n')
    for variable in low_coefs:
        print('Variable: {}\nCoefficient: {}\n'
              .format(variable[1],variable[0]))
    print('************')

    #Print MSE for the train an test set.
    train_mse = mean_squared_error(y_train, pred_y_train)
    test_mse = mean_squared_error(y_test, pred_y_test)
    print('\nTrain MSE: {}\nTest MSE: {}\nDifference:{}\n'
          .format((train_mse),(test_mse),
                  (train_mse-test_mse)))
    
    #Print R^2 against the test data. 
    print('Train R^2: {}'
          .format((r2_score(y_train,pred_y_train))))
    print('CrossValidated R^2: {}'
          .format(np.mean(cross_val_score(model,
                                          ind_train, y_train, 
                                          scoring ='r2', 
                                          cv = crossval))))
    print('Test R^2: {}\n'
          .format(r2_score(y_test,pred_y_test)))
    

    plotdf = pd.DataFrame([])
    plotdf['test_resids'] = pred_y_test-y_test
    plotdf['y_test'] = y_test
    
    sns.jointplot(x = 'y_test', y = 'test_resids',
                  data=plotdf, kind='kde')
    plt.show()
    if save_as != None:
        plt.savefig(save_as);
    
    plt.scatter(pred_y_test, y_test)
    plt.plot(y_test, y_test, color='black', 
             label='Actual Price')
    plt.xlabel('Predicted Price')
    plt.ylabel('Actual Price')
    plt.legend();

In [ ]:
report(model1, X_train, X_test, 
       save_as = 'figures/Model1Resids.png')

Based on the MSE and R-squared, the model seems fairly accurate, with a few problems. Namely it seems to lack predictive value for highly priced homes, and it predicts values into the negative values, which is of course impossible with housing prices. 

The variables floors, bedrooms, and month_sold appear to lack predictive influence. Information from the scatterplots during data exploration suggest that this is the result of the relationship being better modeled by polynomial regression.

The low difference between MSE of the test set and MSE against the training set indicates that this model is not overfit.

Finally, based on the KDE plot of residuals, it is clear that the model could be more generalizable, especially for high value homes. Interaction and polynomial features will likely improve predictive quality of the model. 

# Train the Model
In order to train the model and verify it's validity I will conduct the following steps:

1. Find and include interaction features.
2. Find and include polynomial features.
3. Satisfy Assumptions.
4. Validate Model

## Find and Include Interaction Features

In [ ]:
def plot_interaction(col1, col2):
    """
    Plot the regression lines of variables grouped by
    high and low values. Non-parellel lines show 
    interaction of variables.
    
    Parameters:
    col1: pandas Series. Variable to group by.
    col2: pandas Series. Variable to plot by
    """
    sample = X_train.join(y_train, how='outer')
    
    hisample = (sample.loc[sample[col1]
                           >sample[col1].quantile(.66)])
    midsample = (sample.loc[(sample[col1]
                             >=sample[col1].quantile(.33))
                            |(sample[col1]
                              <=sample[col1].quantile(.66))])
    losample = (sample.loc[sample[col1]
                           <sample[col1].quantile(.33)])
    
    fig, axes = plt.subplots(figsize=(6,3))
    sns.regplot(x=col2, y='price', data=hisample, 
                scatter=False, truncate=True,
                label='High Values of {}'.format(col1))
    sns.regplot(x=col2, y='price', data=midsample, 
                scatter=False, truncate=True,
                label='Middle Values of {}'.format(col1))
    sns.regplot(x=col2, y='price', data=losample, 
                scatter=False, 
                label='Low Values of {}'.format(col1))
    plt.title('Interaction of {} and {}'.format(col1, col2))
    plt.legend()
    plt.show();
    print('*********************\n')

In [ ]:
def find_interactions(n, model, ind_train):
    """
    Returns n most predictive interactions based on low MSE.
    
    Parameters:
    n: int. the number of interactions selected.
    model: LinearRegression() object being tested. 
    ind_train: the independent variables in the training set.
    """
    combos = list(combinations(ind_train.columns, 2))
    print('Testing {} combinations.\n'.format(len(combos)))
    inters = [(0,0,0)]*n
    temp_X = ind_train.loc[:]
    for combo in combos:
        temp_X['interaction']=(ind_train.loc[:, combo[0]]
                               *ind_train.loc[:, combo[1]])
        linreg = LinearRegression()
        model = linreg.fit(temp_X, y_train)
        y_pred = model.predict(temp_X)
        score = round(r2_score(y_train, y_pred),3)
        if score > inters[-1][0]:
            inters.append((score, combo[0], combo[1]))
            inters = sorted(inters, reverse=True)[:n]
    for inter in inters:
        print('R^2 including interaction of {} and {}: {}'
              .format(inter[1], inter[2], inter[0]))
        plot_interaction(inter[1], inter[2])
    fin_inters = [i[1:] for i in inters]
    return fin_inters

In [ ]:
interactions = find_interactions(15, model1, X_train)

In [ ]:
print('Homes south of Seattle: {}'
      .format(data[data['lat']<47.608013]['lat'].count()))
print('Distance of Farthest Southern home: {}'
      .format(47.608013-data['lat'].min()))
print('Distance of Farthest Nothern home: {}'
      .format(data['lat'].max()-47.608013))
data['lat'].describe()

In [ ]:
interactions

In [ ]:
for inter in interactions:
    var = pd.DataFrame(X_train[inter[0]] * X_train[inter[1]])
    linreg = LinearRegression()
    model = linreg.fit(var, y_train)
    y_pred = model.predict(var)
    plt.scatter(var, y_train, alpha=.2)
    plt.scatter(var, y_pred, c='red')
    plt.title('{}*{}'.format(inter[0], inter[1]))
    plt.show()

These interactions illustrate an interesting phenomenon. Despite the general trends, each city seems to have different reactions to the values of the other variables. This is most clear in the chart evaluating the impact of latitude on Bellevue: whereas in most cases the price of a home increases the farther North it is, in Bellevue the trend is reversed. 

Because of this, I will include the following interactions:
- lat and dist_to_Seattle: houses in the middle latitudes drop in price more steeply than others as they gain distance from Seattle. This is likely the result of moving inland rather than north or south along the coast.

- lat and Seattle: houses in the Northernmost set of data lose value if they are in Seattle compared to Southernmost set of data. This indicates that home values are likely higher in southern Seattle than they are in the northern part of the city.

- sqft_living and dist_to_Seattle: Larger houses lose value at a steeper pace as they get farther from Seattle compared to middle sized homes which hold their value better.

- long and Seattle: Houses farther to the West(the coastline) increase their value significantly more by being in Seattle than houses farther East.

- lat and long: lat and long together tell a more complete piture of location.

- lat and Bellevue: Houses at the Southern side of Bellevue are more expensive than houses at the Northern side.

- sqft_living and Seattle: smaller houses increase in value more by being located in Seattle than larger houses.

- long and dist_to_Seattle: middle values of longitude decrease in price at a slower rate as they get farther from Seattle.

In [ ]:
remove = [('grade', 'dist_to_Seattle'),
          ('sqft_living15', 'dist_to_Seattle'),
          ('sqft_living', 'lat'), ('grade', 'lat'),
          ('grade', 'Kent'), ('lat', 'Kent')]
for inter in remove:
    interactions.remove(inter)

In [ ]:
def add_interactions(interactions, ind_train, ind_test):
    """
    Use forward selection based on lowest MSE to select
    and add most predictive interactions to a new model.
    
    Parameters:
    interactions: list of tuples outputed by find_interaction
    function.
    ind_train: independent variables training data.
    ind_test: independent variables test data.
    
    Returns:
    (new_model, new_x_with_interactions)
    """
    additions = interactions
    X_temp_tr = ind_train.loc[:]
    X_best_tr = ind_train.loc[:]
    X_best_t = ind_test.loc[:]
    scores = []
    baseline = 0
    while additions:
        for inter in additions:
            X_temp_tr[inter[0]
                      +' * '
                      +inter[1]]=(X_temp_tr.loc[:, inter[0]]
                                  *X_temp_tr.loc[:, inter[1]])
            linreg = LinearRegression()
            model = linreg.fit(X_temp_tr, y_train)
            y_pred = model.predict(X_temp_tr)
            score = round(r2_score(y_train, y_pred),5)
            scores.append((score, inter[0], inter[1]))
            best = sorted(scores, reverse=True)[0]
            x_temp_tr = X_best_tr.loc[:]
        scores = []
        if best[0] >= baseline:
            additions.remove(best[1:])
            baseline = best[0]
            X_best_tr[best[1]
                      +' * '
                      +best[2]]=(X_temp_tr.loc[:, best[1]]
                                 * X_temp_tr
                                 .loc[:, best[2]])
            X_best_t[best[1]
                     +' * '
                     +best[2]]=(X_best_t.loc[:, best[1]]
                                *X_best_t.loc[:, best[2]])
            linreg = LinearRegression()
            model = linreg.fit(X_best_tr, y_train)
            y_pred = model.predict(X_best_tr)
            t_pred = model.predict(X_best_t)
            print('Interaction Added: {} * {}'
                  .format(best[1], best[2]))
            print('Current R^2: {}'
                  .format(best[0]))
            print('Current Test MSE: {}'
                  .format(round(mean_squared_error
                                (y_test, t_pred),5)))
            print('Current MSE Difference: {}\n'
                  .format(round(best[0]
                          -round(mean_squared_error
                                 (y_test, t_pred),5),5)))
        else:
            print('complete')
            break
    linreg = LinearRegression()
    new_model = linreg.fit(X_best_tr, y_train)
    return(new_model, X_best_tr, X_best_t)

In [ ]:
model2, X_train2, X_test2 = add_interactions(interactions, 
                                             X_train, X_test)

In [ ]:
report(model2, X_train2, X_test2, 
       save_as = 'figures/Model2Resids.png')

## Find and include Polynomial Features
To find polynomial features I will iterate through the data, attempting to find data that suggests a polynomial relationship between price and the independent variable. I will then   

In [ ]:
def get_polynomial_features(ind_train, 
                            max_degree = 3):
    features = []
    for col in ind_train.columns:
        scores = []
        for degree in range(1,max_degree + 1):
            df = pd.DataFrame(ind_train[col])
            poly = PolynomialFeatures(degree)
            X_poly_train = poly.fit_transform(df)
            reg_poly = LinearRegression().fit(X_poly_train,
                                              y_train)
            y_pred = reg_poly.predict(X_poly_train)
            score = round(mean_squared_error(y_train,
                                             y_pred),5)
            scores.append((score, degree, col, df, y_pred))
            best_score = sorted(scores)[0] 
        if best_score[1] > 1:
            print('Factor {} by {}'
                  .format(best_score[2], 
                          best_score[1]))
            plt.scatter(best_score[3], y_train, alpha = .1)
            plt.scatter(best_score[3], best_score[4], 
                        c='red', label=('Predicted Values'))
            plt.legend()
            plt.show()
            features.append((best_score[2], best_score[1]))
    return features 

In [ ]:
poly_feat = get_polynomial_features(X_train2, 
                                    max_degree = 10)

In [ ]:
poly_feat

In [ ]:
def add_polynomials(feats, ind_train, ind_test):
    train = ind_train.loc[:]
    test = ind_test.loc[:]
    for feat in feats:
        train_df = pd.DataFrame(ind_train[feat[0]])
        test_df = pd.DataFrame(ind_test[feat[0]])
        poly = PolynomialFeatures(feat[1])
        poly_train = poly.fit_transform(train_df)
        display(poly_train)
        poly_test =poly.fit_transform(test_df)
        train['{}^{}'.format(feat[0],feat[1])]=poly_train
        test['{}^{}'.format(feat[0],feat[1])]=poly_test
    return train, test

#X_train3, X_test3 = add_polynomials(poly_feat, 
 #                                   X_train2, X_test2)

In [ ]:
#Perform the same function as PolynomialFeatures, but name
# the columns.
X_train3 = X_train2.loc[:]
X_test3 = X_test2.loc[:]
for feat in poly_feat:
    factor = 2
    while factor <= feat[1]:
        poly_train = X_train2[feat[0]]**factor
        poly_test = X_test2[feat[0]]**factor
        X_train3['{}^{}'.format(feat[0], factor)]=poly_train
        X_test3['{}^{}'.format(feat[0], factor)]=poly_test
        factor +=1

In [ ]:
linreg = LinearRegression()
model3 = linreg.fit(X_train3, y_train)

In [ ]:
report(model3, X_train3, X_test3)

In [ ]:
y_pred = model3.predict(X_train3)
len(y_train)
#X_train3['resids']=y_train- y_pred
#X_train3[X_train3['resids']>3]

In [ ]:
X_train3.info()

## Filter by P-values

In [ ]:
def optimize_variables(X,y):
    """
    Forward selects variables by adjusted R^2.
    
    Parameters:
    X: pandas dataframe. independent variables.
    y: pandas series. dependent variable.
    
    Returns:
    Optimized model. 
    """
current_score, best_score = 0, 0
remaining = set(X.columns)
selected = pd.DataFrame()
while remaining and not current_score > best_score:
    scores_with_candidates = []
    for candidate in remaining:
        test_x = selected.join(X[str(candidate)], how='outer')
        score = sm.OLS(y,test_x).fit().rsquared_adj
        scores_with_candidates.append((score,candidate))
    scores_with_candidates.sort()
    best_score, best_candidate = scores_with_candidates.pop()
    if current_score < best_score:
        remaining.remove(best_candidate)
        selected = selected.join(X[str(best_candidate)], how='outer')
        current_score = best_score
print('Selected Columns: {}'.format(selected.columns))
best_vars = sm.add_constant(selected)
return best_vars

In [ ]:
def isolate(col):
    ind_var = pd.DataFrame(X[col]) 
    frozen_var = X.drop(col, axis=1)
    for fro_col in frozen_var.columns:
        frozen_var[fro_col] = frozen_var[fro_col].mean()
    iso_X = ind_var.join(frozen_var, how='outer')
    return iso_X